## Create a recipe database by scrapping the data from a website

In [1]:
from bs4 import BeautifulSoup #scraping
import requests #for HTTP requests
import re #for regular expression
import time 

import pandas as pd
import numpy as np
import spacy


import random

### 1. Defining function to scrape the website "allrecipes.com"

In [15]:
base_url = "https://vegsoc.org/recipes/?_paged="
page=2

r = requests.get(base_url + str(page))

In [3]:
r

<Response [200]>

In [4]:
html = BeautifulSoup(r.content, "html.parser")

In [5]:
recipes_all = []

In [40]:
recipes_all

[{'name': 'Cauilflower and Tofu Bibimbap',
  'url': 'https://vegsoc.org/recipes/bibimbap/'},
 {'name': 'Firecracker Traybake',
  'url': 'https://vegsoc.org/recipes/firecracker-traybake/'},
 {'name': 'Japanese Sharing Platter',
  'url': 'https://vegsoc.org/recipes/japanese-sharing-platter/'},
 {'name': 'Sticky Cider-Glazed Sausage Traybake',
  'url': 'https://vegsoc.org/recipes/sausage-traybake/'},
 {'name': 'Guinness Pie with Colcannon, Thyme Roasted Carrots and Tenderstem Broccoli',
  'url': 'https://vegsoc.org/recipes/st-patricks-day-pie/'},
 {'name': 'Summer Barbecue Paella',
  'url': 'https://vegsoc.org/recipes/summer-paella/'},
 {'name': 'Homemade Barbecue Sauce',
  'url': 'https://vegsoc.org/recipes/homemade-barbecue-sauce/'},
 {'name': 'Grilled Halloumi',
  'url': 'https://vegsoc.org/recipes/grilled-halloumi/'},
 {'name': 'Spicy Potato Wedges',
  'url': 'https://vegsoc.org/recipes/spicy-potato-wedges/'},
 {'name': 'Sweet Potato Burger with Griddled Pineapple and Chilli Salsa',
 

In [146]:
curr____Recipe = BeautifulSoup(requests.get(url = recipes_prod[151]['url']).content, "html.parser")

In [50]:
curr____Recipe_1 = BeautifulSoup(requests.get(url = recipes_all[11]['url']).content, "html.parser")

In [2]:
def getRecipeLink(base_url, totalPage):
    recipes_all = []
    for page in range(1, totalPage):
        r = requests.get(base_url + str(page)) #returns a variable which contains the html doc unstructured
        print(r)
        html = BeautifulSoup(r.content, "html.parser")
        k = 0 #to count recipes per page
        for i in html.findAll('article', {"class": "recipe-snippet"}):
            link = i.find("a", attrs={'href': re.compile("https://vegsoc.org/recipes/")})
            name = i.find('h3', {"class":"recipe-snippet__heading"}).text
            recipes_all.append({"name" : name, "url" : link['href']})
            k+=1
        print("Extracted {} recipes".format(k))
    return recipes_all

In [3]:
def get_ingreds_together(currRecipe):
    ingreds_together = []
    uls = currRecipe.find_all("ul", {"class": "ingredients-list__list no-list"})
    for ul in uls:
        lis = ul.find_all("li", {"class": "ingredients-list__item js-ingredient-item"})
        for li in lis:
            text = li.text
            if isinstance(text, str):
                text = text.strip()
                ingreds_together.append(text)
    return ingreds_together

In [43]:
get_ingreds_together(curr____Recipe)

['440ml good quality vegan cider',
 '1 tsp chopped chives',
 '2 tbsp olive oil',
 '2 tbsp white caster sugar',
 '11/2 tbsp English mustard',
 'Salt and pepper, to taste',
 '500g mini baby new potatoes',
 '1 tbsp olive oil',
 '2 whole garlic bulbs, tops cut off to expose the cloves',
 '6 vegan sausages, left whole or cut into halves',
 '2 red onions, cut into wedges',
 '150g Tenderstem broccoli',
 '2 red apples, cut into wedges',
 '100g vegan hollandaise sauce',
 '2 tbsp gherkins, finely chopped',
 '1 tsp chives, chopped',
 '1 tsp chopped flat-leaf parsley',
 'A handful flat-leaf parsley and torn basil']

In [4]:
def get_ingreds_separately(currRecipe):
    divs = currRecipe.find_all("div", {"class": "ingredients-list__meal-part"})
    if divs is None:
        return "no info"
    try:
        headings = [div.find("h5", {"class": "ingredients-list__meal-part-heading"}).text for div in divs]
    except AttributeError:
        return "no info"
    ingred_lists = [get_ingreds_together(div) for div in divs]
    ingred_lists_dict = dict(zip(headings, ingred_lists))
    return ingred_lists_dict

In [45]:
get_ingreds_separately(curr____Recipe)

{'For the Sticky Cider Glaze': ['440ml good quality vegan cider',
  '1 tsp chopped chives',
  '2 tbsp olive oil',
  '2 tbsp white caster sugar',
  '11/2 tbsp English mustard',
  'Salt and pepper, to taste'],
 'For the Traybake': ['500g mini baby new potatoes',
  '1 tbsp olive oil',
  '2 whole garlic bulbs, tops cut off to expose the cloves',
  '6 vegan sausages, left whole or cut into halves',
  '2 red onions, cut into wedges',
  '150g Tenderstem broccoli',
  '2 red apples, cut into wedges'],
 'For the dipping sauce': ['100g vegan hollandaise sauce',
  '2 tbsp gherkins, finely chopped',
  '1 tsp chives, chopped',
  '1 tsp chopped flat-leaf parsley'],
 'To garnish': ['A handful flat-leaf parsley and torn basil']}

In [5]:
def get_nutritional_metrics(currRecipe):
    nutritional_metrics_names_list = ["calories", "protein", "fat", "saturates", "carbs", "sugar", "fibre", "salt"]
    lis = currRecipe.find_all("li", {"class": "nutritional-metrics-collection__item"})
    nutritional_metrics_values = [li.find("p", {"class": "nutritional-metrics-card__value"}).text.strip() for li in lis]
    nutritional_metrics_keys = [li.find("p", {"class": "nutritional-metrics-card__key"}).text.strip().lower() for li in lis]
    if lis:
        nutritional_info_dict = dict(zip(nutritional_metrics_keys, nutritional_metrics_values))
    else:
        nutritional_info_dict = dict(zip(nutritional_metrics_names_list, ["no info"]*len(nutritional_metrics_names_list)))
    return nutritional_info_dict

In [53]:
get_nutritional_metrics(curr____Recipe)

{'calories': 'no info',
 'protein': 'no info',
 'fat': 'no info',
 'saturates': 'no info',
 'carbs': 'no info',
 'sugar': 'no info',
 'fibre': 'no info',
 'salt': 'no info'}

In [54]:
get_nutritional_metrics(curr____Recipe_1)

{'calories': '796.6 kcal',
 'protein': '23.6g',
 'fat': '30.2g',
 'saturates': '5.2g',
 'carbs': '115.4g',
 'sugar': '2.8g',
 'fibre': '1.38225g'}

In [6]:
def get_keypoints_list(currRecipe):
    ps = currRecipe.find_all("p", {"class": "recipe-key-points__text"})
    keypoints_list = [p.text.strip() for p in ps]
    return keypoints_list

In [57]:
get_keypoints_list(curr____Recipe_1)

['Dairy-Free',
 'Egg-Free',
 'Gluten-Free',
 'Suitable for Freezing',
 'Wheat-Free']

In [58]:
get_keypoints_list(curr____Recipe)

['Dairy-Free', 'Egg-Free', 'Gluten-Free', 'Nut-Free', 'Vegan', 'Wheat-Free']

In [7]:
def get_prep_info_part(currRecipe, info_type):
    p = currRecipe.find("p", {"class": "recipe-meta-snippet recipe-meta-snippet--" + info_type})
    if p is None:
        return "no info"
    text = p.find("span", {"class": "recipe-meta-snippet__text"}).text
    number = re.findall(pattern=r"\d+", string=text)[0]
    return number    

In [81]:
get_prep_info_part(curr____Recipe_1, "time")

'50'

In [8]:
def get_prep_info_dict(currRecipe, list_info_types = ["serves", "prep", "time"]):
    dict_keys = ["servings_number", "preparation_time_minutes", "cooking_time_minutes"]
    prep_info_dict = {}
    for num_info, info_type in enumerate(list_info_types):
        prep_info_dict[dict_keys[num_info]] = get_prep_info_part(currRecipe, info_type)
    return prep_info_dict

In [83]:
get_prep_info_dict(curr____Recipe_1, list_info_types = ["serves", "prep", "time"])

{'servings_number': '4',
 'preparation_time_minutes': '15',
 'cooking_time_minutes': '50'}

In [9]:
def get_secondary_info_dict(currRecipe):
    secondary_info_dict = {"meal_type": "no info",
                          "difficulty": "no info"}
    keys__ = ["meal_type", "difficulty"]
    lis = currRecipe.find_all("li", {"class": "image-split-banner__secondary-info-item"})[:2]
    for num_li, li in enumerate(lis):
        if num_li == 0:
            a = li.find("a", attrs={'href': re.compile("/recipes/")})
            if a:
                text = li.find("a", attrs={'href': re.compile("/recipes/")}).text.strip()
            else:
                text = "no info"
            secondary_info_dict["meal_type"] = text
        else:
            span = li.find("span", {"class": "recipe-meta-snippet__text"})
            if span:
                text = " ".join(span.text.strip().split(" ")[1:])
            else:
                text = "no info"
            secondary_info_dict["difficulty"] = text
    return secondary_info_dict

In [147]:
get_secondary_info_dict(curr____Recipe)

{'meal_type': 'Main', 'difficulty': 'no info'}

In [10]:
def get_recipe_name(currRecipe):
    div = currRecipe.find("div", {"class": "image-split-banner__primary-info"})
    h1 = div.find("h1", {"class": "image-split-banner__heading"})
    recipe_name = h1.text.strip()
    return recipe_name

In [92]:
get_recipe_name(curr____Recipe)

'Sticky Cider-Glazed Sausage Traybake'

In [11]:
def get_instructions(currRecipe):
    lis = currRecipe.find_all("li", {"class": "method-list__item"})
    instructions = [li.text.strip() for li in lis]
    return instructions

In [96]:
get_instructions(curr____Recipe_1)

['Gently fry the onion in a large pan, after 5 mins add the garlic and continue to cook for another 2 minutes minute. Add the turmeric, paprika, ginger and chilli and cook for another 2 minutes.',
 'Add sweet potato, cauliflower, peppers, chickpeas, cashew nuts, stock and coconut milk then bring to the boil and simmer for 20 minutes stirring occasionally. If you would like a thicker sauce mix 1 tsp of cornflour with a little water to make a paste. Add this to the Korma and gently stir for 5 minutes.',
 'Finally add the peas and most of the coriander retaining a little for garnish. Adjust the seasoning and serve with a spoonful of natural yogurt and coriander to garnish.',
 'To make the rice: Gently fry the onion in a large pan. After 5 minutes add the spices and continue to fry for 2 minutes. Add the rice and cover with water. Bring to the boil, and then simmer for 20 minutes until the water has been absorbed. Finally add the peas and cook for another 5 minutes. Depending on the size o

In [16]:
def getEachRecipe(recipesList):
    for ix in range(len(recipesList)):
        url = recipesList[ix]['url']
        r = requests.get(url) #returns a variable which contains the html doc unstructured
        currRecipe = BeautifulSoup(r.content, "html.parser")
        #there are two versions of the website, the old and new have different html structure
        #they require different parsing
        ver = 0
        recipe_name = get_recipe_name(currRecipe)
        recipesList[ix]['name'] = recipe_name
        
        instructions = get_instructions(currRecipe)
        recipesList[ix]['instructions'] = instructions
        
        ingred_one_list = get_ingreds_together(currRecipe)
        recipesList[ix]['ingred_one_list'] = ingred_one_list
        
        ingred_separate_lists = get_ingreds_separately(currRecipe)
        recipesList[ix]['ingred_separate_lists'] = ingred_separate_lists
        
        key_points_list = get_keypoints_list(currRecipe)
        recipesList[ix]['key_points_list'] = key_points_list
               
        nutritional_info = get_nutritional_metrics(currRecipe)
        recipesList[ix]['nutritional_info_dict'] = nutritional_info       

        
        preparation_info_dict = get_prep_info_dict(currRecipe, list_info_types = ["serves", "prep", "time"])
        
        servings_number = preparation_info_dict["servings_number"]
        preparation_time_minutes = preparation_info_dict["preparation_time_minutes"]
        cooking_time_minutes = preparation_info_dict["cooking_time_minutes"]
        
        recipesList[ix]['servings_number'] = servings_number
        recipesList[ix]['preparation_time_minutes'] = preparation_time_minutes
        recipesList[ix]['cooking_time_minutes'] = cooking_time_minutes
        
        secondary_info_dict = get_secondary_info_dict(currRecipe)
        meal_type = secondary_info_dict["meal_type"]
        difficulty = secondary_info_dict["difficulty"]
        
        recipesList[ix]['meal_type'] = meal_type
        recipesList[ix]['difficulty'] = difficulty
        
        print("\r", "Adding {0} ingredients to {1} recipe".format(len(ingred_one_list), ix + 1), end=" ")
    return recipesList

In [134]:
recipes_test = getRecipeLink(base_url, 1)

<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes


In [135]:
data_test = getEachRecipe(recipes_test)

 Adding 6 ingredients to 108 recipe  

In [13]:
import json

In [136]:
with open('vegsoc_test.json', 'w') as fp:
    json.dump(data_test, fp, indent=4)

In [17]:
recipes_prod = getRecipeLink(base_url, 50)

<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [200]>
Extracted 12 recipes
<Response [2

In [61]:
import time

In [65]:
import codecs

In [62]:
start_scrapping = time.time()
data_prod = getEachRecipe(recipes_prod)
end_scrapping = time.time()

duration_scrapping_minutes = (end_scrapping - start_scrapping) / 60
print("Scrapping took {} minutes".format(duration_scrapping_minutes))

 Adding 7 ingredients to 588 recipe Scrapping took 10.446027092138927 minutes


In [64]:
with open('vegsoc_prod.json', 'w', encoding="utf-8") as fp:
    json.dump(data_prod, fp, indent=4, ensure_ascii=False)

In [67]:
with codecs.open("2021-09-19_vegsoc_prod.json", "w", "utf-8") as fp:
    json.dump(data_prod, fp, indent=4, ensure_ascii=False)

In [70]:
with open('vegsoc_prod.json', 'r', encoding="utf-8") as fp_read:
    json_ascii = json.load(fp_read)
    with open("vegsoc_prod_2021-09-19.json", 'w', encoding="utf-8") as fp_write:
        json.dump(json_ascii, fp_write, indent=4, ensure_ascii=False)  